In [0]:
# Importando Bibliotecas
from pyspark.sql import DataFrame, Row
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, NumericType, DateType, BooleanType, TimestampType
from datetime import datetime

# Cria tabela intermediária de vendas

In [0]:
%sql
CREATE OR REPLACE TEMPORARY VIEW inter_vendas AS
WITH temp_order_items AS (
  SELECT DISTINCT 
    order_id,
    -- order_item_id,
    product_id,
    seller_id,
    shipping_limit_date,
    price,
    freight_value,
    row_ingestion_timestamp
  FROM poc_datum.olist.order_items_silver
  WHERE 1 =1 
)

SELECT 
 o.order_id	
,o.customer_id	
,o.order_status	
,o.order_purchase_timestamp	
,o.order_approved_at	
,o.order_delivered_carrier_date	
,o.order_delivered_customer_date	
,o.order_estimated_delivery_date
,o.row_ingestion_timestamp
,oi.product_id	
,oi.seller_id	
,oi.shipping_limit_date	
,oi.price	
,oi.freight_value	
FROM poc_datum.olist.order_silver AS o
INNER JOIN temp_order_items as oi
  ON o.order_id = oi.order_id
-- WHERE o.order_id = 'c642b243037172b9fa789eec22ab713c'


In [0]:
%sql
SELECT DISTINCT order_status FROM inter_vendas

order_status
shipped
canceled
approved
invoiced
delivered
unavailable
processing


In [0]:
# Seleciona o banco de dados 'olist' para uso
table_name = 'inter_sales'
spark.sql('USE olist')
spark.sql(f'DROP TABLE IF EXISTS inter_sales')

df = spark.sql('SELECT * FROM inter_vendas')

# Grava o DataFrame no formato Parquet
destination_path = f'dbfs:/FileStore/datum/silver/olistbr-brazilian-ecommerce/{table_name}'
df.write.mode('overwrite').parquet(destination_path)

df.write.format("delta").mode('overwrite').saveAsTable(table_name)